<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Parsing Output

Let's set up a Chat Model:

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI
api_key = open("C://Users//Marcial//Desktop//desktop_openai.txt").read()
model = ChatOpenAI(openai_api_key=api_key)

/Users/marci/opt/miniconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## List Parsing 

In [2]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [3]:
output_parser = CommaSeparatedListOutputParser()

In [4]:
format_instructions = output_parser.get_format_instructions()

In [5]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [6]:
reply = "one, two, three"
output_parser.parse("one, two, three")

['one', 'two', 'three']

In [7]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [9]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions())

ChatPromptValue(messages=[HumanMessage(content='give me 5 characteristics of dogs Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`')])

In [10]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [11]:
result = model.invoke(request)

In [12]:
result.content

'Loyal, friendly, energetic, playful, protective'

In [13]:
# Convert to desired output:
output_parser.parse(result.content)

['Loyal', 'friendly', 'energetic', 'playful', 'protective']

## Datetime Parser 

In [14]:
from langchain.output_parsers import DatetimeOutputParser

In [15]:
output_parser = DatetimeOutputParser()

In [16]:
print(output_parser.get_format_instructions())

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1811-08-10T13:30:58.867859Z, 0716-08-11T10:11:47.627529Z, 1728-12-13T02:34:20.386803Z

Return ONLY this string, no other words!


In [17]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [18]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [19]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1566-01-25T00:18:45.739922Z, 1191-04-08T23:34:15.821274Z, 1483-12-11T12:57:09.951782Z

Return ONLY this string, no other words!


In [20]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [21]:
result = model.invoke(request,temperature=0)

In [22]:
# Careful with this, it sometimes will include extra information!
result.content

'1865-12-06T00:00:00.000000Z'

In [23]:
result.content

'1865-12-06T00:00:00.000000Z'

In [24]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

---

# Methods to Fix Parsing Issues

## Auto-Fix Parser

In [25]:
from langchain.output_parsers import OutputFixingParser

output_parser = DatetimeOutputParser()

misformatted = result.content

In [26]:
misformatted

'1865-12-06T00:00:00.000000Z'

In [27]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)

In [28]:
new_parser.parse(misformatted)

datetime.datetime(1865, 12, 6, 0, 0)

____
### Fixing via System Prompt:

In [29]:
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [30]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [31]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

System: You always reply to questions only in datetime patterns.
Human: When was the 13th Amendment ratified in the US?
Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 0878-01-27T13:27:08.832493Z, 2028-04-22T18:12:45.217964Z, 0771-11-18T02:53:24.751667Z

Return ONLY this string, no other words!


In [32]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [33]:
result = model.invoke(request,temperature=0)

In [34]:
result.content

'1865-12-06T00:00:00.000000Z'

In [35]:
output_parser.parse(result.content)

datetime.datetime(1865, 12, 6, 0, 0)

Be careful! This could technically be construed as wrong? The full details from Wikipedia:

    27th state to ratify was Georgia: December 6, 1865

    Having been ratified by the legislatures of three-fourths of the states (27 of the 36 states, including those that had been in rebellion), Secretary of State Seward, on December 18, 1865, certified that the Thirteenth Amendment had become valid, to all intents and purposes, as a part of the Constitution.

You also have the issue of states leaving the union, which complicates what a full ratification means at that time. It kind of depends what is meant by the word "ratified"!

## Pydantic JSON Parser
You should also be aware of OpenAI's own JSON offerings (which are still quite new at this time!): https://platform.openai.com/docs/guides/gpt/function-calling


In [36]:
#pip install pydantic

In [37]:
from langchain.output_parsers import PydanticOutputParser

In [38]:
from pydantic import BaseModel, Field

In [39]:
class Scientist(BaseModel):
    
    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [40]:
query = 'Name a famous scientist and a list of their discoveries' 

In [41]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [42]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of a Scientist", "title": "Name", "type": "string"}, "discoveries": {"description": "Python list of discoveries", "items": {}, "title": "Discoveries", "type": "array"}}, "required": ["name", "discoveries"]}
```


In [47]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query="Tell me about a famous scientist")

output = model.invoke(_input.to_string())

parser.parse(output.content)

Scientist(name='Albert Einstein', discoveries=['Theory of Relativity', 'Photoelectric Effect', 'Mass-Energy Equivalence'])